In [1]:
import pandas as pd

df_train = pd.read_csv('./dataset/kaggle/train.csv')
df_test = pd.read_csv('./dataset/kaggle/test.csv')

In [2]:
print(df_train.head(5))

   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...


拿掉HTML

In [3]:
print(df_train.loc[0,'Page content'])

<html><head><div class="article-info"> <span class="byline basic">Clara Moskowitz</span> for <a href="/publishers/space-com/">Space.com</a> <time datetime="Wed, 19 Jun 2013 15:04:30 +0000">2013-06-19 15:04:30 UTC</time> </div></head><body><h1 class="title">NASA's Grand Challenge: Stop Asteroids From Destroying Earth</h1><figure class="article-image"><img class="microcontent" data-fragment="lead-image" data-image="http://i.amz.mshcdn.com/I7b9cUsPSztew7r1WT6_iBLjflo=/950x534/2013%2F06%2F19%2Ffe%2FDactyl.44419.jpg" data-micro="1" data-url="http://mashable.com/2013/06/19/nasa-grand-challenge-asteroid/" src="http://i.amz.mshcdn.com/I7b9cUsPSztew7r1WT6_iBLjflo=/950x534/2013%2F06%2F19%2Ffe%2FDactyl.44419.jpg"/></figure><article data-channel="world"><section class="article-content"> <p>There may be killer asteroids headed for Earth, and NASA has decided to do something about it. The space agency announced a new "Grand Challenge" on June 18 to find all dangerous space rocks and figure out how t

In [11]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# 讀取資料
data = pd.read_csv('./dataset/kaggle/train.csv')

# 定義一個函數來清理HTML內容
def clean_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text()

# 應用清理函數到前10筆資料
data.loc[:9, 'cleaned_content'] = data.loc[:9, 'Page content'].apply(clean_html)

# 提取標題
data.loc[:9, 'title'] = data.loc[:9, 'Page content'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').find('h1').get_text() if BeautifulSoup(x, 'html.parser').find('h1') else None
)

# 提取作者的函數
def extract_author(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    article_info = soup.head.find('div', {'class': 'article-info'})

    # 提取作者名稱
    author_name = article_info.find('span', {'class': 'author_name'})
    if author_name:
        author = author_name.get_text()
    elif article_info.span:
        author = article_info.span.string
    elif article_info.a:
        author = article_info.a.string
    else:
        author = "none"

    # 清理作者名稱
    author = re.sub(r'\s+', ' ', author.strip().lower())
    if author.startswith('by '):
        author = author[3:]  # 去掉前綴 'by '
    author = re.sub(r'&.*;', '&', author.replace(' and ', ' & '))

    # 分離作者
    author_list = []
    if author.find(',') == -1:
        author_list = re.split(r'\s*&\s*', author)
    else:
        authors = re.split(r'\s*,\s*', author)
        if authors[-1].find('&') == -1 or len(authors[-1].split('&')[-1].strip().split()) > 3:
            author_list.append(authors[0])
        else:
            author_list += authors[:-1]
            author_list += re.split(r'\s*&\s*', authors[-1])
    author = ' '.join([re.sub(r'\s+', '_', a) for a in author_list])

    return author

# 提取作者
data.loc[:9, 'author'] = data.loc[:9, 'Page content'].apply(extract_author)

# 提取主題
def extract_topic(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    topic_tag = soup.find('footer', class_='article-topics')
    if topic_tag:
        return ', '.join([a.get_text().strip() for a in topic_tag.find_all('a')])
    return "none"

# 提取主題
data.loc[:9, 'topic'] = data.loc[:9, 'Page content'].apply(extract_topic)

# 提取時間
def extract_time(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    time_tag = soup.find('time')
    return time_tag['datetime'] if time_tag else "none"

# 提取時間
data.loc[:9, 'time'] = data.loc[:9, 'Page content'].apply(extract_time)

# 如果需要，你可以選擇在這裡儲存處理後的資料或執行其他操作
# 例如：選擇需要的列
result = data[['Id', 'title', 'author', 'topic', 'time', 'cleaned_content']]

# 顯示前10筆結果
print(result.head(10))


   Id                                              title  \
0   0  NASA's Grand Challenge: Stop Asteroids From De...   
1   1  Google's New Open Source Patent Pledge: We Won...   
2   2  Ballin': 2014 NFL Draft Picks Get to Choose Th...   
3   3        Cameraperson Fails Deliver Slapstick Laughs   
4   4  NFL Star Helps Young Fan Prove Friendship With...   
5   5  The Underdog Internet Providers Head to Washin...   
6   6  6-Second Vine Video Is the Funniest Parody of ...   
7   7  Bill Cosby is still planning to perform his co...   
8   8  Vending Machine Uses Cloud Technology to Perso...   
9   9  Online Subscribers Up, Advertising Still Down ...   

                    author                                              topic  \
0          clara_moskowitz  Asteroid, Asteroids, challenge, Earth, Space, ...   
1         christina_warren  Apps and Software, Google, open source, opn pl...   
2                sam_laird  Entertainment, NFL, NFL Draft, Sports, Television   
3              

   Id                                              title author  \
0   0  NASA's Grand Challenge: Stop Asteroids From De...   None   

                                     cleaned_content  
0   Clara Moskowitz for Space.com 2013-06-19 15:0...  
